In [1]:
import pandas as pd
import numpy as np
from numpy import array

In [2]:
def get_coefficients( File ) :
    with open(File, 'r') as model :
        for line in model :
            performance = line.split(',')[0]
            coefficients = line.split(',')[1:]
    coefficients = [float(i) for i in coefficients]
    return coefficients

In [16]:
model_coefficients_01 = get_coefficients( '../../Step04_model_building/MODEL/MODEL_001.txt' )

In [17]:
len( model_coefficients_01 )

113

In [18]:
import os 
LEVEL = '1000x5/'
# LEVEL = '1000x6/'
# LEVEL = '1000x7/'
# LEVEL = '1000x8/'
# LEVEL = '1000x9/'
# LEVEL = '1000x10/'
PATH_MODEL_DATA = './MODEL_Data/' + LEVEL 
PATH_MODEL_RANK = './MODEL_Data/RANK/'
PATH_MODEL_RESULT = './MODEL_Data/' + LEVEL + 'Result/'
FILE_LIST_LEN = 100

In [19]:
# type(w) : numpy.ndarray
# w.shape : 114 x 1

def score(coefficients, answer0, answer1):
    sub = answer0 - answer1
    score = sub.dot(coefficients)
    if score >= 0:
        return 1
    else :
        return -1

In [20]:
# test0
# answer_i.shape
import operator
import collections, math

def Rank_List( AnswerList, coefficients ):
    Score_Map = collections.defaultdict(lambda : 0)
    for i in range( len(AnswerList) - 1 ):

        for j in range(i+1, len(AnswerList) ):

            #print (i, j)

            answer_i = array( AnswerList.iloc[[i]].values.tolist()[0] )
            answer_j = array( AnswerList.iloc[[j]].values.tolist()[0] )

            key_i = 'A_' + str(i) + '_score'
            key_j = 'A_' + str(j) + '_score'

            value_i = 0
            value_j = 0
            value_i += score(coefficients, answer_i, answer_j) 
            value_j -= score(coefficients, answer_i, answer_j)

            Score_Map[key_i] += value_i
            Score_Map[key_j] += value_j
    
    return Score_Map
    # Sorted_scores = sorted(Score_Map.items(), key=operator.itemgetter(1), reverse=True)
    # return Sorted_scores  

In [21]:
def writeDict(Dict, filename, sep):
    with open(filename, "w") as f:
#        for key in dict.keys():            
#            f.write(str(key) + ":" + str(dict[key]) + "\n")
        for key, value in sorted( Dict.items() ):
            f.write( str(key) + sep + str(value) + '\n' )

def writeList(List, filename) :
    with open(filename, 'w') as f:
        for i in range( len(List) ) :
            # print List[i][0]
            # print List[i][1]
            f.write( str(List[i][0]) + ':' + str(List[i][1]) + '\n')
            
def readDict(filename, sep):
    with open(filename, "r") as f:
        dict = {}
        for line in f:
            values = line.split(sep)
            key = values[0]
            value = int(values[1])
            #value = values[1].split(',')
            #dict[values[0]] = {int(x) for x in values[1:len(values)]}
            dict[key] = value
        return dict 

In [22]:
# Answer_Rank = []
import operator

def update_RankList( data_path, file_list_len = FILE_LIST_LEN, \
                    path_out = PATH_MODEL_RANK, coefficients = model_coefficients_01) :

    for i in range(file_list_len) :
        input_file_name = 'test_' + str(i) + '.csv'
        # print input_file_name
        test_i = pd.read_csv(data_path + input_file_name , header=None)
        test_i = test_i.fillna(0)

        output_file_name_01 = 'test_' + str(i) + '_dict.txt'
        output_file_name_02 = 'test_' + str(i) + '_sorted.txt'

        # Rank test_i with given coefficients
        Ai_List = Rank_List(test_i, coefficients)
        Ai_Sorted = sorted(Ai_List.items(), key=operator.itemgetter(1), reverse=True)

        writeDict( Ai_List  , path_out  + output_file_name_01, ':')
        writeList( Ai_Sorted, path_out  + output_file_name_02 )
    # print Rank_List(test_i) 

In [23]:
# row 代表读取文件的第几行
# rank 代表要匹配这一行的答案
# 判断 sorted file 某一行的答案是否是 answer0
def hit( sorted_file , row, rank = 0, path_in = PATH_MODEL_RANK) :
    # print type(rank)
    hit = 0
    with open( path_in + sorted_file, 'r') as f:
        
        while( int(row) > 0) :
            # print row
            f.readline()
            row = row - 1
        
        answer = f.readline()
        # print answer
        answer_key = answer.split(':')[0]
        # print answer_key
        answer_rank = answer_key.split('_')[1]
        # print answer_rank
        if int(rank) == int(answer_rank) :
                hit = 1
        else :
            hit = 0
    return hit

In [24]:
def calculate_p_at_k( fout, file_list_len = FILE_LIST_LEN, path_out = PATH_MODEL_RESULT ) :    
    answer0_hit_0 = 0
    answer0_hit_1 = 0
    answer0_hit_2 = 0
    answer0_hit_3 = 0
    answer0_hit_4 = 0

    p_at_k = {}

    for i in range(file_list_len) :

        test_sorted = 'test_' + str(i) + '_sorted.txt'

        answer0_hit_0 += hit( test_sorted, 0)
        answer0_hit_1 += hit( test_sorted, 1)
        answer0_hit_2 += hit( test_sorted, 2)
        answer0_hit_3 += hit( test_sorted, 3)
        answer0_hit_4 += hit( test_sorted, 4) 
    
    p_at_k['p@1'] = answer0_hit_0
    p_at_k['p@2'] = p_at_k['p@1'] + answer0_hit_1
    p_at_k['p@3'] = p_at_k['p@2'] + answer0_hit_2
    p_at_k['p@4'] = p_at_k['p@3'] + answer0_hit_3
    p_at_k['p@5'] = p_at_k['p@4'] + answer0_hit_4
    
    writeDict( p_at_k  , path_out  + fout , ':')
    return p_at_k

In [25]:
def ave_p_at_k(path_in = PATH_MODEL_RESULT, k = 1) :
    ave_p_at_k = 0.
    file_list = os.listdir(path_in)
    # print file_list
    for fname in file_list :
        # p_at_k is a list 
        p_at_k = readDict(path_in + fname, ':')
        key = 'p@' + str(k)
        ave_p_at_k += p_at_k[key]
    
    return ave_p_at_k / len(file_list)

In [28]:
for i in range(10) : 
    fname = 'Final_Test_0' + str(i) 
    data_path = PATH_MODEL_DATA + fname + '/'
    file_list = os.listdir(data_path)
    # print len(file_list)
    
    update_RankList( data_path )
    
    fout = fname + '.csv'
    p_at_k = calculate_p_at_k( fout )
    print p_at_k

{'p@3': 80, 'p@2': 66, 'p@1': 48, 'p@5': 94, 'p@4': 88}
{'p@3': 84, 'p@2': 70, 'p@1': 43, 'p@5': 96, 'p@4': 94}
{'p@3': 77, 'p@2': 68, 'p@1': 46, 'p@5': 92, 'p@4': 87}
{'p@3': 77, 'p@2': 68, 'p@1': 47, 'p@5': 93, 'p@4': 86}
{'p@3': 82, 'p@2': 72, 'p@1': 56, 'p@5': 94, 'p@4': 90}
{'p@3': 72, 'p@2': 65, 'p@1': 43, 'p@5': 88, 'p@4': 77}
{'p@3': 70, 'p@2': 63, 'p@1': 49, 'p@5': 92, 'p@4': 81}
{'p@3': 77, 'p@2': 71, 'p@1': 47, 'p@5': 95, 'p@4': 88}
{'p@3': 81, 'p@2': 72, 'p@1': 49, 'p@5': 92, 'p@4': 86}
{'p@3': 73, 'p@2': 64, 'p@1': 45, 'p@5': 89, 'p@4': 86}


In [29]:
ave_p_at_1 = ave_p_at_k()
ave_p_at_2 = ave_p_at_k(k=2)
ave_p_at_3 = ave_p_at_k(k=3)
ave_p_at_4 = ave_p_at_k(k=4)
ave_p_at_5 = ave_p_at_k(k=5)

ave_p_at_k = {}
ave_p_at_k['ave_p_at_1'] = ave_p_at_1
ave_p_at_k['ave_p_at_2'] = ave_p_at_2
ave_p_at_k['ave_p_at_3'] = ave_p_at_3
ave_p_at_k['ave_p_at_4'] = ave_p_at_4
ave_p_at_k['ave_p_at_5'] = ave_p_at_5
writeDict( ave_p_at_k  , PATH_MODEL_RESULT  + 'ave_result.csv' , ':')

print ave_p_at_1
print ave_p_at_2
print ave_p_at_3
print ave_p_at_4
print ave_p_at_5

47.3
67.9
77.3
86.3
92.5
